### Function

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os
import math


def arrowedLine_shift(s_cos, s_sin, w2, h2):

    ####
    # Warning Adjust the size of the arrow window
    # w<w2, h<h2
    ####

    w=500
    h=500
    line_len = 200

    #center
    pt1 = (w//2, h//2)

    # line style
    color = (0,0,0)
    thickness = 20

    # create img with line
    img = np.zeros((h, w, 3), np.uint8)
    img.fill(255)

    if s_cos == -2:
        cv.circle(img, pt1, 63, (0,0,255), -1)       #значит нет смещения
        back = np.zeros((w2, h2, 3), np.uint8)
        back.fill(255)
        x = round((w2 - w) / 2)
        y = round((h2 - h) / 2)

        # mask for img1
        result = back.copy()
        result[x:x+w, y:y+h] = img
        return result

    dx = int(s_cos * line_len)
    dy = int(s_sin * line_len)

    pt2 = (pt1[0]-dx, pt1[1]-dy)
    cv.arrowedLine(img, pt1, pt2, color, thickness)

    # resize
    back = np.zeros((w2, h2, 3), np.uint8)
    back.fill(255)
    x = round((w2 - w) / 2)
    y = round((h2 - h) / 2)

    # mask for img1
    result = back.copy()
    result[x:x+w, y:y+h] = img


    return result


def descript_shift(dx, dy):
    dist = math.sqrt(dx*dx + dy*dy)
    if dist > 1e-5:
        s_cos = dx/dist
        s_sin = dy/dist
        return s_cos, s_sin
    else:
        return -2, -2   #no shift



def get_image(source_p, target_p):
    img1 = cv.imread(source_p)
    img2  = cv.imread(target_p)
    print('size input: {}, size target: {}'.format(img1.shape, img2.shape))

    height1, width1 = img1.shape[:2]
    height2, width2 = img2.shape[:2]


    min_height = min(height1, height2)
    min_width = min(width1, width2)


    img = cv.resize(img1, (min_width, min_height))
    target_img = cv.resize(img2, (min_width, min_height))
    print('REsize input: {}, size target: {}'.format(img.shape, target_img.shape))

    return img, target_img



def find_shift(source_p, target_p, detecter, matcher, trashold=0.6, show=True, name_out='demos'):
    img, target_img = get_image(source_p, target_p)

    file_name = os.path.basename(source_p)
    file = os.path.splitext(file_name)
    out_p =  name_out + '/' + file[0].replace('query','result') + '.jpg'
    print(out_p)

    if not os.path.isdir(name_out):
     os.mkdir(name_out)

    keypoints, descriptors = detecter.detectAndCompute(img, None)
    target_keypoints, target_descriptors = detecter.detectAndCompute(target_img, None)

    matches = matcher.knnMatch(descriptors, target_descriptors, k=2)

    good_matches = []
    for m1, m2 in matches:
        if m1.distance < trashold*m2.distance:
            good_matches.append([m1])

    print("size of good mathes: ", len(good_matches))

    matchesMask = [[0,0] for i in range(len(matches))]
    for i,(m1, m2) in enumerate (matches):
        if m1.distance < trashold * m2.distance:
            matchesMask[i] = [1,0]

    for i, match in enumerate(good_matches[:400]):
        pt1 = keypoints[good_matches[i][0].queryIdx].pt                         #SOURCE
        pt2= target_keypoints[good_matches[i][0].trainIdx].pt                   #TARGET

        img = cv.circle(img, (int(pt1[0]), int(pt1[1])), 5, (0, 0, 255), -1)    # red   SOURCE
        img = cv.circle(img, (int(pt2[0]), int(pt2[1])), 5, (0, 255, 15), -1)   # green TARGET

        img = cv.line(img, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])),(0,15, 55),3)

    df_x=0
    df_y=0
    dx_m = []
    dy_m = []
    for match in good_matches:
        x1, y1 = keypoints[match[0].queryIdx].pt
        x2, y2 = target_keypoints[match[0].trainIdx].pt

        dx_m.append(x2 - x1)
        dy_m.append(y2 - y1)

        df_x += x2 - x1
        df_y += y2 - y1

    #print("D_x:", df_x)
    #print("D_y:", df_y)
    #print("D_x_max:", max(dx_m))
    #print("D_y_max:", max(dy_m))

    s_cos, s_sin = descript_shift(df_x, df_y)
    #print(s_cos, s_sin)

    #make output
    img1 = img
    img2 = arrowedLine_shift(s_cos, s_sin, img1.shape[0], img1.shape[1])
    img3 = target_img

    result = cv.hconcat([img1, img3, img2])
    cv.imwrite(out_p, result)
    return s_cos, s_sin



def get_list_files(path, extensions):
    names_file = []
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            split_tup = os.path.splitext(file)
            if split_tup[1] in extensions:
                names_file.append(os.path.join(path, file))
    return names_file


In [ ]:
#create sift
sift = cv.SIFT_create()

# FLANN parameters faster then BFMatcher
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv.FlannBasedMatcher(index_params,search_params)

### Make Test

In [ ]:
result={}
PATH_TO_QUERY = 'data/'
for item in get_list_files(PATH_TO_QUERY, ['.jpeg','.jpg']):
    file_name = os.path.basename(item)
    if 'target' not in file_name:
         s_cos, s_sin = find_shift(
             source_p = PATH_TO_QUERY + file_name,
             target_p = PATH_TO_QUERY + file_name.replace('query', 'target'),
             detecter=sift,
             matcher=flann,
             show=False,
             trashold = 0.5,
             name_out='results'
          )
         result[file_name] = (s_cos, s_sin)

size input: (4032, 3024, 3), size target: (2403, 1284, 3)
REsize input: (2403, 1284, 3), size target: (2403, 1284, 3)
results/result_1.jpg
size of good mathes:  130
size input: (4032, 3024, 3), size target: (4032, 3024, 3)
REsize input: (4032, 3024, 3), size target: (4032, 3024, 3)
results/result_16.jpg
size of good mathes:  241
size input: (4032, 3024, 3), size target: (4032, 3024, 3)
REsize input: (4032, 3024, 3), size target: (4032, 3024, 3)
results/result_20.jpg
size of good mathes:  1640
size input: (4032, 3024, 3), size target: (4032, 3024, 3)
REsize input: (4032, 3024, 3), size target: (4032, 3024, 3)
results/result_21.jpg
size of good mathes:  6276
size input: (4032, 3024, 3), size target: (4032, 3024, 3)
REsize input: (4032, 3024, 3), size target: (4032, 3024, 3)
results/result_17.jpg
size of good mathes:  298
size input: (500, 500, 3), size target: (500, 500, 3)
REsize input: (500, 500, 3), size target: (500, 500, 3)
results/result_9.jpg
size of good mathes:  22
size input: (

### VIEW RESULT

In [ ]:
print(result)

{'query_1.jpeg': (0.9996942026704204, 0.02472854923266672), 'query_16.jpeg': (0.9975564350450438, -0.06986529109810859), 'query_20.jpeg': (0.07750090344576717, -0.9969922817981541), 'query_21.jpeg': (-0.26930199413551903, 0.9630557802924153), 'query_17.jpeg': (-0.21968122457225553, 0.9755717090867458), 'query_9.jpg': (0.9999999926510844, -0.00012123461229362528), 'query_8.jpg': (0.7118080508644479, 0.7023740447400912), 'query_11.jpg': (-0.7064304038728131, 0.7077825121349738), 'query_26.jpeg': (-0.5544910824527669, 0.8321896655693092), 'query_10.jpg': (-0.7070221523552253, -0.7071913998904291), 'query_12.jpg': (-0.9999999682441397, 0.0002520153161397545), 'query_13.jpg': (0.0008155130896813118, 0.999999667469145), 'query_27.jpeg': (0.5570033417633812, 0.8305102511495123), 'query_24.jpeg': (-0.9995968684853355, -0.028391909310766704), 'query_5.jpeg': (0.7160878565188452, -0.6980101587700895), 'query_4.jpeg': (0.8591796083415588, 0.5116741156342047), 'query_25.jpeg': (-0.0348066598177781